#4. Matrix Factorization(MF) 기반 추천

||메모리 기반 알고리즘|모델 기반 알고리즘|
|---|---|---|
|설명|메모리에 있는 데이터를 계산해서 <br> 추천하는 방식|데이터로부터 미리 모델을 구성 후, <br> 필요 시 추천하는 방식|
|특징|개별 사용자 데이터 집중|전체 사용자 패턴 집중|
|장점|원래 데이터에 충실하게 사용|대규모 데이터에 빠르게 반응|
|단점|대규모 데이터에 느리게 반응|모델 생성 과정 오래 걸림|
|예시|CF 기반 추천 알고리즘|MF 기반 추천 알고리즘, 딥러닝|

## 4.1. Matrix Factorization(MF) 방식의 원리
* Matrix Factorization : 행렬 요인화/분해
* 평가, 사용자, 아이템으로 구성된 하나의 행렬을, 두 개의 행렬로 분해
* R ≈ P × Q.T = R^ (예상 평점)
  * Rating matrix R - M×N 차원
  * User latent matrix 사용자와 사용자 잠재 요인 행렬 P - M×K 차원
  * Item latent matrix 아이템과 아이템 잠재 요인 행렬 Q - N×K 차원
* CF에서는 사용자와 아이템, 평점으로 이루어진 full-matrix R 이용

## 4.2. SGD(Stochastic Gradient Decent)를 사용한 MF 알고리즘
 * SGD를 사용해 MF의 P와 Q 행렬을 구하는 게 최종 목표  

### 4.2.1. MF 알고리즘 개념적 설명
> 1. 잠재 요인 개수 K 선택
> 2. P, Q 행렬 초기화  
>
> [ 반복 ]  
> > 3. 예측 평점 R_hat(= P×Q.T) 계산
> > 4. 실제 R과 R_hat 간 오차 계산 및 P, Q 수정 (오차 감소 위함)
> > → 가장 중요한 단계
> > 5. 기준 오차 도달 확인

* MF의 핵심 : P와 Q 잘 분해하기
  * 주어진 사용자와 아이템의 관계를 잘 설명할 수 있도록  

### 4.2.2. SGD : Stochastic Gradient Decent
* 예측 오차를 줄이기위한 P, Q 업데이트
  * 예측 오차 제곱의 편미분 값 사용
  * 학습률(learning rate) α 알파 활용
* Overfitting 과적합 방지
  * 정규화 고려
    * 정규화 항(Regulation term) 추가
    * 정교화 계수 β
  * 경향성 고려
    * 사용자와 아이템의 경향성 문제
      * 전체 평균 b
      * 전체 평균을 제거한 후 사용자 i의 평가 경향 bu[i]
        * 사용자 i 평균과 전체 평균의 차이
      * 전체 평균을 제거한 후 아이템 j의 평가 경향 bi[j]
        * 아이템 j의 평균과 전체 평균의 차이
    * CF에서는 사용자와 아이템 별로 평가 경향이 한 번에 계산되었는데  
    MF에서는 계산할 때마다 오차를 최소화하도록 bu[i]와 bi[j] 계속 업데이트




## 4.3. SGD를 사용한 MF 기본 알고리즘

In [4]:
import os
import pandas as pd
import numpy as np

base_src = "drive/MyDrive/RecoSys/python-recosys/Data"
u_data_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_data_src, sep = "\t", names = r_cols, encoding = "latin-1")
ratings = ratings[["user_id", "movie_id", "rating"]].astype(int)

In [7]:
class MF() :
  # hyper_params : 알파나 베타 값 등 딕셔너리로
  def __init__(self, ratings, hyper_params) :
    # 데이터프레임 형식으로 전달된 평점 넘파이 배열로 바꾸기
    self.R = np.array(ratings)
    self.num_users, self.num_items = np.shape(self.R)
    self.K = hyper_params["K"] # 잠재 요인 개수
    self.alpha = hyper_params["alpha"] # 학습률
    self.beta = hyper_params["beta"] # 정교화 계수
    self.iterations = hyper_params["iterations"] # SGD 얼만큼 반복
    self.verbose = hyper_params["verbose"] # 학습 과정 중간에 출력할 것인지 여부를 판단하는 플래그 변수

  # P와 Q를 이용해 RMSE를 계산하는 함수
  def rmse(self) :
    xs, ys = self.R.nonzero() # 0이 아닌 요소의 인덱스 반환
    self.predictions = [] # 나중에 prediction과 error를 담을 리스트 변수 초기화
    self.errors = []

    for x, y in zip(xs, ys) :
      prediction = self.get_prediction(x, y) # 사용자 x 아이템 y 에서 평점예측치 계산하는 함수
      self.predictions.append(prediction)
      # 실제값과 예측값의 차이를 오차값으로 설정
      self.errors.append(self.R[x, y] - prediction)
    self.predictions = np.array(self.predictions)
    self.errors = np.array(self.errors)

    return np.sqrt(np.mean(self.errors**2))

  # 학습 메소드
  def train(self) :
    # P와 Q 우선 난수값으로 초기화
    # mean을 지정하지 않으면 디폴트로 0
    # 표준 편차 sacle을 1/잠재변수개수 로 지정
    self.P = np.random.normal(scale = 1./self.K, size = (self.num_users, self.K))
    self.Q = np.random.normal(scale = 1./self.K, size = (self.num_items, self.K))

    # 사용자 평가 경향
    self.b_u = np.zeros(self.num_users)

    # 아이템
    self.b_d = np.zeros(self.num_items)

    # 평점의 전체 평균
    self.b = np.mean(self.R[self.R.nonzero()])

    # SGD를 적용할 대상 설정
    rows, columns = self.R.nonzero()
    # 평점의 인덱스와 평점을 리스트로 만들어서 저장
    self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

    # SGD가 한 번 실행될 때마다 RMSE가 얼마나 계산되는지 기록하는 리스트
    training_process = []
    for i in range(self.iterations) :
      # 다양한 시작점에서 SGD 적용
      # 데이터의 순서에 따라 모델의 학습 경로가 영향을 받을 수 있기 때문에, 데이터를 무작위로 섞는 것은 중요
      np.random.shuffle(self.samples)
      self.sgd()
      rmse = self.rmse()
      training_process.append((i+1, rmse))

      # SGD 학습 과정을 중간에 출력할 건지 여부
      if self.verbose :
        if (i+1) % 10 == 0 :
          print("Iteration : %d ; train RMSE = %.4f" %(i+1, rmse))
    return training_process

  # 평점 예측값 구하는 함수
  # 아이템 j에 대한 사용자 i의 평점 예측치
  def get_prediction(self, i, j) :
    # R_hat
    # 전체 평점 + 사용자 평가 경향 + 아이템에 대한 평가 경향 + (사용자 i의 요인 값과 아이템 j 요인의 행렬 연산)
    predriction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
    return predriction

  # 최적의 P, Q, B_U, B_D 구하기 위한 과정
  def sgd(self) :
    for i, j, r in self.samples :
      prediction = self.get_prediction(i, j)
      # 실제 평점과 비교해 오차 계산
      e = (r - prediction)

      # 사용자 평가 경향 계산 및 업데이트
      self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
      # 아이템 평가 경향 계산 및 업데이트
      self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

      # 행렬 P 계산 및 업데이트
      self.P[i, :] += self.alpha * ((e * self.Q[j, :]) - (self.beta * self.P[i, :]))
      # 행렬 Q 계산 및 업데이트
      self.Q[j, :] += self.alpha * ((e * self.P[i, :]) - (self.beta * self.Q[j, :]))

In [ ]:
R_temp = ratings.pivot(index = "user_id", columns = "movie_id", values = "rating").fillna(0)
hyper_params = {
    "K" : 30,
    "alpha" : 0.001,
    "beta" : 0.02,
    "iterations" : 100,
    "verbose" : True
    }
mf = MF(R_temp, hyper_params)
train_process = mf.train()

Iteration : 10 ; train RMSE = 0.9585
Iteration : 20 ; train RMSE = 0.9374
Iteration : 30 ; train RMSE = 0.9281
Iteration : 40 ; train RMSE = 0.9225
Iteration : 50 ; train RMSE = 0.9183
Iteration : 60 ; train RMSE = 0.9143
Iteration : 70 ; train RMSE = 0.9095
Iteration : 80 ; train RMSE = 0.9030
Iteration : 90 ; train RMSE = 0.8937
